In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)

In [2]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error

import math
import pandas as pd
import operator
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from scipy import stats
import random
import copy

from scipy.stats import norm, kurtosis
from sklearn.metrics import make_scorer

from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

In [3]:
import tensorflow as tf
import keras
import keras.backend as K
from keras import layers, models, optimizers, initializers, regularizers, constraints, losses
from keras.models import Sequential, Model, load_model, Input
from keras.layers import (Dense, Concatenate, BatchNormalization, Activation, Add,
                          concatenate, Dropout, AlphaDropout, Reshape, Layer, Multiply, Lambda)
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.losses import binary_crossentropy,  kullback_leibler_divergence, mean_squared_error

def mish(x):
    return x*K.tanh(K.softplus(x))

def decay(epoch, steps=100):
    initial_lrate = 1e-3
    drop = 0.9
    epochs_drop = 50
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    lrate = max(lrate, 5e-5)
    return lrate

es = EarlyStopping(patience=25, restore_best_weights=True)
lrs = LearningRateScheduler(decay, verbose=2)

Using TensorFlow backend.


In [4]:
target = pd.read_csv('./data/SolarPV_Elec_Problem.csv', header=None)
data = pd.read_csv('./data/features3.csv')#.iloc[0:-2,:] # 자료가 21시까지
# sub = pd.read_csv('./data/제출양식_복원값.xlsx')



In [5]:
def grap_year(x):
    y, _, _ = x.split('-')
    return int(y)

def grap_date(x):
    y = x[5:10].replace('-','')
    return y

def grap_hour(x):
    return x[11:13]

def custom_loss(y_true, y_pred):
    res = 0
    cnt = 1
    for i in range(0, len(y_true), 24):
        yt = y_true[i:i+24]
        yp = y_pred[i:i+24]
        a = np.abs(yt-yp)
        c = 113
        S = np.sum(yt)
        res += np.sum(a*yt/(S*c))
        cnt += 1
    return res/cnt

class Lookahead(keras.optimizers.Optimizer):
    """The lookahead mechanism for optimizers.
    Default parameters follow those provided in the original paper.
    # Arguments
        optimizer: An existed optimizer.
        sync_period: int > 0. The synchronization period.
        slow_step: float, 0 < alpha < 1. The step size of slow weights.
    # References
        - [Lookahead Optimizer: k steps forward, 1 step back]
          (https://arxiv.org/pdf/1907.08610v1.pdf)
    """

    def __init__(self, optimizer, sync_period=5, slow_step=0.5, **kwargs):
        super(Lookahead, self).__init__(**kwargs)
        self.optimizer = keras.optimizers.get(optimizer)
        with K.name_scope(self.__class__.__name__):
            self.sync_period = K.variable(sync_period, dtype='int64', name='sync_period')
            self.slow_step = K.variable(slow_step, name='slow_step')

    @property
    def lr(self):
        return self.optimizer.lr

    @lr.setter
    def lr(self, lr):
        self.optimizer.lr = lr

    @property
    def learning_rate(self):
        return self.optimizer.learning_rate

    @learning_rate.setter
    def learning_rate(self, learning_rate):
        self.optimizer.learning_rate = learning_rate

    @property
    def iterations(self):
        return self.optimizer.iterations

    def get_updates(self, loss, params):
        sync_cond = K.equal((self.iterations + 1) // self.sync_period * self.sync_period, (self.iterations + 1))
        slow_params = {p.name: K.variable(K.get_value(p), name='sp_{}'.format(i)) for i, p in enumerate(params)}
        update_names = ['update', 'update_add', 'update_sub']
        original_updates = [getattr(K, name) for name in update_names]
        setattr(K, 'update', lambda x, new_x: ('update', x, new_x))
        setattr(K, 'update_add', lambda x, new_x: ('update_add', x, new_x))
        setattr(K, 'update_sub', lambda x, new_x: ('update_sub', x, new_x))
        self.updates = self.optimizer.get_updates(loss, params)
        for name, original_update in zip(update_names, original_updates):
            setattr(K, name, original_update)
        slow_updates = []
        for i, update in enumerate(self.updates):
            if isinstance(update, tuple):
                name, x, new_x, adjusted = update + (update[-1],)
                update_func = getattr(K, name)
                if name == 'update_add':
                    adjusted = x + new_x
                if name == 'update_sub':
                    adjusted = x - new_x
                if x.name not in slow_params:
                    self.updates[i] = update_func(x, new_x)
                else:
                    slow_param = slow_params[x.name]
                    slow_param_t = slow_param + self.slow_step * (adjusted - slow_param)
                    slow_updates.append(K.update(slow_param, K.switch(
                        sync_cond,
                        slow_param_t,
                        slow_param,
                    )))
                    self.updates[i] = K.update(x, K.switch(
                        sync_cond,
                        slow_param_t,
                        adjusted,
                    ))
        slow_params = list(slow_params.values())
        self.updates += slow_updates
        self.weights = self.optimizer.weights + slow_params
        return self.updates

    def get_config(self):
        config = {
            'optimizer': keras.optimizers.serialize(self.optimizer),
            'sync_period': int(K.get_value(self.sync_period)),
            'slow_step': float(K.get_value(self.slow_step)),
        }
        base_config = super(Lookahead, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        optimizer = keras.optimizers.deserialize(config.pop('optimizer'))
        return cls(optimizer, **config)

In [6]:
target['year'] = target[0].map(grap_year)
target['md'] = target[0].map(grap_date)
target['hour'] = target[0].map(grap_hour)
target = target.drop(0, axis=1)

In [7]:
target = target.fillna(-10)
y = target.groupby(['year', 'md', 'hour']).sum()[1].values

In [8]:
y1 = y[:5136] # 0701~0130
y2 = y[5160:6552] # 0201~0330
y3 = y[6600:8016] # 0401~0530
y4 = y[8064:] # 0530~0630

val_y1 = y[5112:5136]
val_y2 = y[6552:6576]
val_y3 = y[8016:8040]

val_y = np.hstack([val_y1, val_y2, val_y3])

In [9]:
tr_y1 = y[24:5136-24]
tr_X1 = y[:5112-24]

val_y1 = y[5112:5136]
val_X1 = y[5112-24:5136-24]

tr_y2 = y[5184:6576-24]
tr_X2 = y[5160:6552-24]

val_y2 = y[6552:6576]
val_X2 = y[6552-24:6552]

tr_y3 = y[6624:8016]
tr_X3 = y[6600:8016-24]

val_y3 = y[8016:8040]
val_X3 = y[8016-24:8016]

tr_y4 = y[8088:8760]
tr_X4 = y[8064:8760-24]

In [10]:
len(tr_X2)/24

57.0

In [11]:
train_X = np.concatenate([tr_X1, tr_X2, tr_X3, tr_X4])
train_y = np.concatenate([tr_y1, tr_y2, tr_y3, tr_y4])
validation_X = np.concatenate([val_X1,  val_X2, val_X3])
validation_y = np.concatenate([val_y1, val_y2, val_y3])

tr_X = []
for i in range(0, len(train_X), 24):
    tr_X.append(train_X[i:i+24])
tr_X = np.array(tr_X)

tr_y = []
for i in range(0, len(train_y), 24):
    tr_y.append(train_y[i:i+24])
tr_y = np.array(tr_y)

val_y = []
for i in range(0, len(validation_y), 24):
    val_y.append(validation_y[i:i+24])
val_y= np.array(val_y)

val_X = []
for i in range(0, len(validation_X), 24):
    val_X.append(validation_X[i:i+24])
val_X= np.array(val_X)

In [12]:
val_X.shape, tr_X.shape

((3, 24), (355, 24))

In [13]:
def AE():
    activation=mish
    nn = Sequential()
    nn.add(Dense(16))
#     nn.add(BatchNormalization())
    nn.add(Activation(activation))
    nn.add(Dense(8))
    nn.add(Activation('sigmoid'))
    nn.add(Dense(16))
#     nn.add(BatchNormalization())
    nn.add(Activation(activation))
    nn.add(Dense(24))
    
    return nn

In [14]:
a = AE()
a.compile(loss='mse', optimizer=Lookahead(optimizers.Adam()))

In [15]:
a.fit(tr_X, tr_y,
     epochs=2000,
     validation_split=0.15,
     shuffle=True,
     callbacks=[es, lrs])

Train on 301 samples, validate on 54 samples
Epoch 1/2000

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
301/301 [==============================] - 0s 1ms/step - loss: 1008.8743 - val_loss: 1369.6661
Epoch 2/2000

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
301/301 [==============================] - 0s 100us/step - loss: 1004.3994 - val_loss: 1361.7658
Epoch 3/2000

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
301/301 [==============================] - 0s 100us/step - loss: 997.2522 - val_loss: 1351.7094
Epoch 4/2000

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
301/301 [==============================] - 0s 96us/step - loss: 987.1496 - val_loss: 1340.4004
Epoch 5/2000

Epoch 00005: LearningRateScheduler setting learning rate to 0.001.
301/301 [==============================] - 0s 106us/step - loss: 980.2023 - val_loss: 1331.9273
Epoch 6/2000

Epoch 00006: LearningRateScheduler setting learning rate 

Epoch 47/2000

Epoch 00047: LearningRateScheduler setting learning rate to 0.001.
301/301 [==============================] - 0s 103us/step - loss: 623.8433 - val_loss: 877.4834
Epoch 48/2000

Epoch 00048: LearningRateScheduler setting learning rate to 0.001.
301/301 [==============================] - 0s 103us/step - loss: 617.3697 - val_loss: 868.8130
Epoch 49/2000

Epoch 00049: LearningRateScheduler setting learning rate to 0.001.
301/301 [==============================] - 0s 100us/step - loss: 610.8122 - val_loss: 860.2297
Epoch 50/2000

Epoch 00050: LearningRateScheduler setting learning rate to 0.0009000000000000001.
301/301 [==============================] - 0s 100us/step - loss: 604.5492 - val_loss: 852.7122
Epoch 51/2000

Epoch 00051: LearningRateScheduler setting learning rate to 0.0009000000000000001.
301/301 [==============================] - 0s 103us/step - loss: 598.8348 - val_loss: 845.1999
Epoch 52/2000

Epoch 00052: LearningRateScheduler setting learning rate to 0.000900

301/301 [==============================] - 0s 106us/step - loss: 426.4931 - val_loss: 613.0258
Epoch 90/2000

Epoch 00090: LearningRateScheduler setting learning rate to 0.0009000000000000001.
301/301 [==============================] - 0s 103us/step - loss: 422.8960 - val_loss: 608.1530
Epoch 91/2000

Epoch 00091: LearningRateScheduler setting learning rate to 0.0009000000000000001.
301/301 [==============================] - 0s 113us/step - loss: 419.3429 - val_loss: 603.3390
Epoch 92/2000

Epoch 00092: LearningRateScheduler setting learning rate to 0.0009000000000000001.
301/301 [==============================] - 0s 105us/step - loss: 415.9317 - val_loss: 598.4699
Epoch 93/2000

Epoch 00093: LearningRateScheduler setting learning rate to 0.0009000000000000001.
301/301 [==============================] - 0s 103us/step - loss: 412.5100 - val_loss: 593.6786
Epoch 94/2000

Epoch 00094: LearningRateScheduler setting learning rate to 0.0009000000000000001.
301/301 [==========================

301/301 [==============================] - 0s 100us/step - loss: 316.6059 - val_loss: 457.8840
Epoch 132/2000

Epoch 00132: LearningRateScheduler setting learning rate to 0.0008100000000000001.
301/301 [==============================] - 0s 100us/step - loss: 314.6368 - val_loss: 454.9861
Epoch 133/2000

Epoch 00133: LearningRateScheduler setting learning rate to 0.0008100000000000001.
301/301 [==============================] - 0s 100us/step - loss: 312.7198 - val_loss: 452.0646
Epoch 134/2000

Epoch 00134: LearningRateScheduler setting learning rate to 0.0008100000000000001.
301/301 [==============================] - 0s 100us/step - loss: 310.7377 - val_loss: 449.2934
Epoch 135/2000

Epoch 00135: LearningRateScheduler setting learning rate to 0.0008100000000000001.
301/301 [==============================] - 0s 100us/step - loss: 308.8230 - val_loss: 446.5094
Epoch 136/2000

Epoch 00136: LearningRateScheduler setting learning rate to 0.0008100000000000001.
301/301 [=====================

301/301 [==============================] - 0s 103us/step - loss: 254.6643 - val_loss: 365.2847
Epoch 174/2000

Epoch 00174: LearningRateScheduler setting learning rate to 0.0007290000000000002.
301/301 [==============================] - 0s 96us/step - loss: 253.6407 - val_loss: 363.5789
Epoch 175/2000

Epoch 00175: LearningRateScheduler setting learning rate to 0.0007290000000000002.
301/301 [==============================] - 0s 100us/step - loss: 252.5701 - val_loss: 361.9516
Epoch 176/2000

Epoch 00176: LearningRateScheduler setting learning rate to 0.0007290000000000002.
301/301 [==============================] - 0s 100us/step - loss: 251.5163 - val_loss: 360.4172
Epoch 177/2000

Epoch 00177: LearningRateScheduler setting learning rate to 0.0007290000000000002.
301/301 [==============================] - 0s 103us/step - loss: 250.5348 - val_loss: 358.7314
Epoch 178/2000

Epoch 00178: LearningRateScheduler setting learning rate to 0.0007290000000000002.
301/301 [======================

...

301/301 [==============================] - 0s 100us/step - loss: 163.2889 - val_loss: 231.2123
Epoch 517/2000

Epoch 00517: LearningRateScheduler setting learning rate to 0.0003486784401000001.
301/301 [==============================] - 0s 106us/step - loss: 163.2410 - val_loss: 231.2288
Epoch 518/2000

Epoch 00518: LearningRateScheduler setting learning rate to 0.0003486784401000001.
301/301 [==============================] - 0s 103us/step - loss: 163.2077 - val_loss: 231.1388
Epoch 519/2000

Epoch 00519: LearningRateScheduler setting learning rate to 0.0003486784401000001.
301/301 [==============================] - 0s 106us/step - loss: 163.1661 - val_loss: 231.1992
Epoch 520/2000

Epoch 00520: LearningRateScheduler setting learning rate to 0.0003486784401000001.
301/301 [==============================] - 0s 103us/step - loss: 163.1350 - val_loss: 231.2082
Epoch 521/2000

Epoch 00521: LearningRateScheduler setting learning rate to 0.0003486784401000001.
301/301 [=====================

301/301 [==============================] - 0s 103us/step - loss: 162.2495 - val_loss: 230.5517
Epoch 559/2000

Epoch 00559: LearningRateScheduler setting learning rate to 0.0003138105960900001.
301/301 [==============================] - 0s 100us/step - loss: 162.2452 - val_loss: 230.6418
Epoch 560/2000

Epoch 00560: LearningRateScheduler setting learning rate to 0.0003138105960900001.
301/301 [==============================] - 0s 96us/step - loss: 162.2072 - val_loss: 230.6301
Epoch 561/2000

Epoch 00561: LearningRateScheduler setting learning rate to 0.0003138105960900001.
301/301 [==============================] - 0s 96us/step - loss: 162.1877 - val_loss: 230.6636
Epoch 562/2000

Epoch 00562: LearningRateScheduler setting learning rate to 0.0003138105960900001.
301/301 [==============================] - 0s 103us/step - loss: 162.1588 - val_loss: 230.6070
Epoch 563/2000

Epoch 00563: LearningRateScheduler setting learning rate to 0.0003138105960900001.
301/301 [=======================

Epoch 601/2000

Epoch 00601: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [==============================] - 0s 136us/step - loss: 161.4460 - val_loss: 229.9959
Epoch 602/2000

Epoch 00602: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [==============================] - 0s 116us/step - loss: 161.4527 - val_loss: 230.0228
Epoch 603/2000

Epoch 00603: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [==============================] - 0s 100us/step - loss: 161.4343 - val_loss: 230.0243
Epoch 604/2000

Epoch 00604: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [==============================] - 0s 103us/step - loss: 161.4747 - val_loss: 230.0716
Epoch 605/2000

Epoch 00605: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [==============================] - 0s 96us/step - loss: 161.4056 - val_loss: 229.9957
Epoch 606/2000

Epoch 00606: Le

301/301 [==============================] - 0s 103us/step - loss: 160.8625 - val_loss: 229.8733
Epoch 644/2000

Epoch 00644: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [==============================] - 0s 103us/step - loss: 160.8343 - val_loss: 229.8840
Epoch 645/2000

Epoch 00645: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [==============================] - 0s 100us/step - loss: 160.8291 - val_loss: 229.8623
Epoch 646/2000

Epoch 00646: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [==============================] - 0s 103us/step - loss: 160.7883 - val_loss: 229.8408
Epoch 647/2000

Epoch 00647: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [==============================] - 0s 100us/step - loss: 160.8095 - val_loss: 229.8449
Epoch 648/2000

Epoch 00648: LearningRateScheduler setting learning rate to 0.0002824295364810001.
301/301 [=====================

301/301 [==============================] - 0s 103us/step - loss: 160.3046 - val_loss: 229.7557
Epoch 686/2000

Epoch 00686: LearningRateScheduler setting learning rate to 0.0002541865828329001.
301/301 [==============================] - 0s 96us/step - loss: 160.2805 - val_loss: 229.7328
Epoch 687/2000

Epoch 00687: LearningRateScheduler setting learning rate to 0.0002541865828329001.
301/301 [==============================] - 0s 103us/step - loss: 160.2832 - val_loss: 229.6829
Epoch 688/2000

Epoch 00688: LearningRateScheduler setting learning rate to 0.0002541865828329001.
301/301 [==============================] - 0s 100us/step - loss: 160.2818 - val_loss: 229.7175
Epoch 689/2000

Epoch 00689: LearningRateScheduler setting learning rate to 0.0002541865828329001.
301/301 [==============================] - 0s 100us/step - loss: 160.2648 - val_loss: 229.7679
Epoch 690/2000

Epoch 00690: LearningRateScheduler setting learning rate to 0.0002541865828329001.
301/301 [======================

In [16]:
q = a.predict(val_X)

In [17]:
custom_loss(val_y.flatten(), q.flatten())

0.14325110655155882

In [18]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def Denseblock(n):
    def f(x):
        activation=mish
        x = Dense(n)(x)
        x = BatchNormalization()(x)
        x = Activation(activation)(x)
        return x
    return f

In [19]:
class VAE:
    def __init__(self, d_dim, latent_dim):
        K.clear_session()
        self.d_dim = d_dim
        self.latent_dim = latent_dim
        self.hist = []
        self.enc, self.dec, self.vae = self.build_vae()

    def build_vae(self):
        enc_input = Input(shape = (self.d_dim[1], ))
        
        x = Denseblock(16)(enc_input)
        x = Denseblock(8)(x)
        # mapping process
        z_mean = Dense(self.latent_dim)(x)
        z_log_var = Dense(self.latent_dim)(x)
        enc_out = Lambda(sampling, output_shape=(self.latent_dim, ))([z_mean, z_log_var])
        
        dec_input = Input(shape = (self.latent_dim, ))
        x = Denseblock(8)(dec_input)
        x = Denseblock(16)(x)
        
        dec_out = Dense(self.d_dim[1], activation='sigmoid')(x)
        
        enc = Model(enc_input, enc_out, name='encoder')
        dec = Model(dec_input, dec_out, name='decoder')
        vae = Model(enc_input, dec(enc(enc_input)), name='vae')
        
        def vae_loss(x, x_decoded_mean):
            xent_loss = binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
            mse = mean_squared_error(x, x_decoded_mean)
            return xent_loss + kl_loss + mse
        
        vae.compile(optimizer=Lookahead(optimizers.Adam()), loss=vae_loss)
        
        return enc, dec, vae     
    
    # or fit manually
    def train(self, x, y, epochs=10, batch_size=1024):
        for e in range(epochs+1):
            idx = np.random.randint(0, len(x), batch_size)
            tr_X = x[idx]
            tr_y = x[idx]

            loss = self.vae.train_on_batch(tr_X, tr_y)
            self.hist.append(loss)


In [20]:
b = VAE(tr_X.shape, 2)

In [21]:
b.vae.fit(tr_X/113, tr_y/113,
       epochs=2000,
         validation_split=0.2,
          shuffle=True,
         callbacks=[es, lrs])

Train on 284 samples, validate on 71 samples
Epoch 1/2000

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
284/284 [==============================] - 1s 4ms/step - loss: 1.2907 - val_loss: 0.8510
Epoch 2/2000

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
284/284 [==============================] - 0s 313us/step - loss: 1.1184 - val_loss: 0.8343
Epoch 3/2000

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
284/284 [==============================] - 0s 310us/step - loss: 1.0015 - val_loss: 0.8171
Epoch 4/2000

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
284/284 [==============================] - 0s 306us/step - loss: 0.9212 - val_loss: 0.8098
Epoch 5/2000

Epoch 00005: LearningRateScheduler setting learning rate to 0.001.
284/284 [==============================] - 0s 296us/step - loss: 0.8615 - val_loss: 0.7946
Epoch 6/2000

Epoch 00006: LearningRateScheduler setting learning rate to 0.001.
284/284 [=======

284/284 [==============================] - 0s 317us/step - loss: 0.3640 - val_loss: 0.4117
Epoch 49/2000

Epoch 00049: LearningRateScheduler setting learning rate to 0.001.
284/284 [==============================] - 0s 299us/step - loss: 0.3603 - val_loss: 0.4114
Epoch 50/2000

Epoch 00050: LearningRateScheduler setting learning rate to 0.0009000000000000001.
284/284 [==============================] - 0s 306us/step - loss: 0.3574 - val_loss: 0.4065
Epoch 51/2000

Epoch 00051: LearningRateScheduler setting learning rate to 0.0009000000000000001.
284/284 [==============================] - 0s 296us/step - loss: 0.3579 - val_loss: 0.4131
Epoch 52/2000

Epoch 00052: LearningRateScheduler setting learning rate to 0.0009000000000000001.
284/284 [==============================] - 0s 310us/step - loss: 0.3526 - val_loss: 0.4080
Epoch 53/2000

Epoch 00053: LearningRateScheduler setting learning rate to 0.0009000000000000001.
284/284 [==============================] - 0s 303us/step - loss: 0.3504

Epoch 92/2000

Epoch 00092: LearningRateScheduler setting learning rate to 0.0009000000000000001.
284/284 [==============================] - 0s 312us/step - loss: 0.3070 - val_loss: 0.3742
Epoch 93/2000

Epoch 00093: LearningRateScheduler setting learning rate to 0.0009000000000000001.
284/284 [==============================] - 0s 313us/step - loss: 0.3056 - val_loss: 0.3742
Epoch 94/2000

Epoch 00094: LearningRateScheduler setting learning rate to 0.0009000000000000001.
284/284 [==============================] - 0s 310us/step - loss: 0.3059 - val_loss: 0.3725
Epoch 95/2000

Epoch 00095: LearningRateScheduler setting learning rate to 0.0009000000000000001.
284/284 [==============================] - 0s 306us/step - loss: 0.3057 - val_loss: 0.3716
Epoch 96/2000

Epoch 00096: LearningRateScheduler setting learning rate to 0.0009000000000000001.
284/284 [==============================] - 0s 306us/step - loss: 0.3050 - val_loss: 0.3704
Epoch 97/2000

Epoch 00097: LearningRateScheduler setti

Epoch 00135: LearningRateScheduler setting learning rate to 0.0008100000000000001.
284/284 [==============================] - 0s 338us/step - loss: 0.2940 - val_loss: 0.3604
Epoch 136/2000

Epoch 00136: LearningRateScheduler setting learning rate to 0.0008100000000000001.
284/284 [==============================] - 0s 306us/step - loss: 0.2931 - val_loss: 0.3602
Epoch 137/2000

Epoch 00137: LearningRateScheduler setting learning rate to 0.0008100000000000001.
284/284 [==============================] - 0s 306us/step - loss: 0.2920 - val_loss: 0.3600
Epoch 138/2000

Epoch 00138: LearningRateScheduler setting learning rate to 0.0008100000000000001.
284/284 [==============================] - 0s 303us/step - loss: 0.2912 - val_loss: 0.3584
Epoch 139/2000

Epoch 00139: LearningRateScheduler setting learning rate to 0.0008100000000000001.
284/284 [==============================] - 0s 317us/step - loss: 0.2919 - val_loss: 0.3589
Epoch 140/2000

Epoch 00140: LearningRateScheduler setting learnin

284/284 [==============================] - 0s 303us/step - loss: 0.2861 - val_loss: 0.3549
Epoch 179/2000

Epoch 00179: LearningRateScheduler setting learning rate to 0.0007290000000000002.
284/284 [==============================] - 0s 300us/step - loss: 0.2865 - val_loss: 0.3550
Epoch 180/2000

Epoch 00180: LearningRateScheduler setting learning rate to 0.0007290000000000002.
284/284 [==============================] - 0s 299us/step - loss: 0.2850 - val_loss: 0.3537
Epoch 181/2000

Epoch 00181: LearningRateScheduler setting learning rate to 0.0007290000000000002.
284/284 [==============================] - 0s 303us/step - loss: 0.2852 - val_loss: 0.3543
Epoch 182/2000

Epoch 00182: LearningRateScheduler setting learning rate to 0.0007290000000000002.
284/284 [==============================] - 0s 303us/step - loss: 0.2854 - val_loss: 0.3543
Epoch 183/2000

Epoch 00183: LearningRateScheduler setting learning rate to 0.0007290000000000002.
284/284 [==============================] - 0s 317u

284/284 [==============================] - 0s 299us/step - loss: 0.2823 - val_loss: 0.3500
Epoch 222/2000

Epoch 00222: LearningRateScheduler setting learning rate to 0.0006561000000000001.
284/284 [==============================] - 0s 306us/step - loss: 0.2823 - val_loss: 0.3512
Epoch 223/2000

Epoch 00223: LearningRateScheduler setting learning rate to 0.0006561000000000001.
284/284 [==============================] - 0s 303us/step - loss: 0.2822 - val_loss: 0.3508
Epoch 224/2000

Epoch 00224: LearningRateScheduler setting learning rate to 0.0006561000000000001.
284/284 [==============================] - 0s 320us/step - loss: 0.2827 - val_loss: 0.3514
Epoch 225/2000

Epoch 00225: LearningRateScheduler setting learning rate to 0.0006561000000000001.
284/284 [==============================] - 0s 306us/step - loss: 0.2824 - val_loss: 0.3519
Epoch 226/2000

Epoch 00226: LearningRateScheduler setting learning rate to 0.0006561000000000001.
284/284 [==============================] - 0s 306u

284/284 [==============================] - 0s 315us/step - loss: 0.2804 - val_loss: 0.3477
Epoch 266/2000

Epoch 00266: LearningRateScheduler setting learning rate to 0.00059049.
284/284 [==============================] - 0s 303us/step - loss: 0.2796 - val_loss: 0.3486
Epoch 267/2000

Epoch 00267: LearningRateScheduler setting learning rate to 0.00059049.
284/284 [==============================] - 0s 331us/step - loss: 0.2803 - val_loss: 0.3485
Epoch 268/2000

Epoch 00268: LearningRateScheduler setting learning rate to 0.00059049.
284/284 [==============================] - 0s 313us/step - loss: 0.2804 - val_loss: 0.3482
Epoch 269/2000

Epoch 00269: LearningRateScheduler setting learning rate to 0.00059049.
284/284 [==============================] - 0s 310us/step - loss: 0.2799 - val_loss: 0.3474
Epoch 270/2000

Epoch 00270: LearningRateScheduler setting learning rate to 0.00059049.
284/284 [==============================] - 0s 299us/step - loss: 0.2802 - val_loss: 0.3480
Epoch 271/2000

284/284 [==============================] - 0s 327us/step - loss: 0.2790 - val_loss: 0.3477
Epoch 312/2000

Epoch 00312: LearningRateScheduler setting learning rate to 0.000531441.
284/284 [==============================] - 0s 313us/step - loss: 0.2789 - val_loss: 0.3463
Epoch 313/2000

Epoch 00313: LearningRateScheduler setting learning rate to 0.000531441.
284/284 [==============================] - 0s 317us/step - loss: 0.2789 - val_loss: 0.3473
Epoch 314/2000

Epoch 00314: LearningRateScheduler setting learning rate to 0.000531441.
284/284 [==============================] - 0s 296us/step - loss: 0.2796 - val_loss: 0.3477
Epoch 315/2000

Epoch 00315: LearningRateScheduler setting learning rate to 0.000531441.
284/284 [==============================] - 0s 313us/step - loss: 0.2786 - val_loss: 0.3468
Epoch 316/2000

Epoch 00316: LearningRateScheduler setting learning rate to 0.000531441.
284/284 [==============================] - 0s 305us/step - loss: 0.2788 - val_loss: 0.3464
Epoch 317

In [22]:
w = b.vae.predict(val_X/113)*113
custom_loss(val_y.flatten(), w.flatten())

0.15663910454867103